In [1]:
using Unitful
include("../src/AutoSolver.jl")

Main.AutoSolver

In [2]:
# include("../../AstroKit.jl/src/AstroKit.jl")
include("../../SciBase.jl/src/SciBase.jl")

Main.physics

In [3]:
# # Displacement
# disp(x0,x) = x-x0
# dispConstAcc1(v0,a,t) = v0*t+1/2*a*t^2
# dispConstAcc2(v0,v,t) = ((v+v0)/2)*t
# # Velocity
# avgVel(x0,x,t0,t) = (x-x0)/(t-t0)
# velConstAcc1(v0,a,t) = v0+a*t
# velConstAcc2(x0,x,v0,a) = sqrt(v0^2+2*a*(x-x0))
# # Acceleration
# accConst(x0,x,v0,v) = (v^2-v0^2)/(2*(x-x0))

In [4]:
# # ve = 10u"km/s"
# # mi = 200u"kg"
# # mf = 80u"kg"
# # f = 2.4u"GHz"
# x0 = 1.0u"m"
# x = 2.0u"m"
# v0 = 1.0u"m/s"
# v = 2.0u"m/s"

# mods = Module[]
# AutoSolver.submodules(physics, mods)
# # mods = [Main]
# # mods = [AstroKit]

# outputs = AutoSolver.autoSolveOutputs(mods)


In [ ]:
# mods = Module[]
# funcs = Dict()
# AutoSolver.submodules(physics, mods)
# AutoSolver.moduleDict(Main.physics.kinematics, funcs)
# mods

In [5]:
# out_d, in_d = AutoSolver.varExplodeDict(funcs)
# inp_map, out_map = AutoSolver.crossVarMap(in_d,out_d)
# inp_map

In [6]:
# import Pkg; Pkg.add("XLSX")

In [7]:
using XLSX

# Create Spreadsheet Template

In [17]:
function autoSolveTemplate(mod::Module;filePath::String="",overwrite=false)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    mod_name = AutoSolver.modName(mod)
    
    if isfile("$(filePath)AS_$(mod_name).xlsx") && overwrite == false
        println("ERROR: $(filePath)AS_$(mod_name).xlsx already exists")
        return
    elseif !isdir(filePath) && filePath != ""
        mkdir(filePath)
    end

    XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="w") do xf
        XLSX.addsheet!(xf)
        XLSX.addsheet!(xf)
        sh1 = xf[1]
        sh2 = xf[2]
        sh3 = xf[3]
        XLSX.rename!(sh1, "Input Map")
        XLSX.rename!(sh2, "Inputs")
        XLSX.rename!(sh3, "Outputs")

        sh1[1,1] = "Module"
        sh1[1,2] = "Function"
        sh1[1,3] = "Input Variable"
        sh1[1,4] = "Input Map"

        sh2[1,1] = "Input Variable"
        sh2[1,2] = "Input"
        sh2[1,3] = "Units"

        sh3[1,1] = "Module"
        sh3[1,2] = "Function"
        sh3[1,3] = "Output"
        sh3[1,4] = "Units"

        r1 = 2
        r2 = 2
        r3 = 2
        new_input = 0
        
        for submod in mods
            funcs = Dict()
            AutoSolver.moduleDict(submod, funcs)
            submod_name = AutoSolver.modName(submod)

            for func in funcs 
                sh1[r1,1] = submod_name
                sh1[r1,2] = string(func[1])

                for i = 1:length(func[2])
                    sh1[r1,3] = string(func[2][i])
                    sh1[r1,4] = string(func[2][i])
                    r1+=1

                    for j = 2:r2
                        if string(sh2[j,1]) == string(func[2][i])
                            new_input = 0
                            break
                        else
                            new_input = 1
                        end
                    end
                    if new_input == 1
                        sh2[r2,1] = string(func[2][i])
                        r2+=1
                    end
                end
            end

            for func in funcs      
                sh3[r3,1] = submod_name
                sh3[r3,2] = string(func[1])
                sh3[r3,3] = "-"
                sh3[r3,4] = "-"
                r3+=1
            end
        end
    end
end

autoSolveTemplate (generic function with 1 method)

In [18]:
autoSolveTemplate(physics, filePath="C:/Users/ses/Documents/AutoSolver/", overwrite=false)

ERROR: C:/Users/ses/Documents/AutoSolver/AS_physics.xlsx already exists


# Update Spreadsheet Mapping

In [19]:
function inputMapFromXLSX(xf::XLSX.XLSXFile)
    sh = xf["Input Map"]
    inp_map = Dict()
    
    for (i,r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        if !haskey(inp_map, r[3]); inp_map[r[3]] = []; end
        if !(r[4] in inp_map[r[3]]); push!(inp_map[r[3]],r[4]); end
    end

    return inp_map
end

inputMapFromXLSX (generic function with 1 method)

In [20]:
function inputsFromXLSX(xf::XLSX.XLSXFile)
    sh = xf["Inputs"]
    
    for (i,r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        if ismissing(@eval $(r[2])); continue; end
        if ismissing(@eval $(r[3]))
            @eval $(Symbol(r[1])) = (@eval $(r[2]))
        else
            @eval $(Symbol(r[1])) = (@eval $(r[2])) * (@eval @u_str $(r[3]))
        end
    end
end

inputsFromXLSX (generic function with 1 method)

In [21]:
function outputToXLSX(xf::XLSX.XLSXFile, outputs::Dict)
    sh = xf["Outputs"]
  
    for (i, r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        func = Symbol(r[2])
        if haskey(outputs,func)
            sh[i,3] = string(ustrip(outputs[func]))
            sh[i,4] = string(unit(outputs[func]))
        end
    end
end

outputToXLSX (generic function with 1 method)

In [26]:
inp_map = Dict()
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics
XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="rw") do xf
    inp_map = inputMapFromXLSX(xf)
    inputsFromXLSX(xf)
end

mods = Module[]
AutoSolver.submodules(mod, mods)
# mods = [Main]
# mods = [AstroKit]

outputs = AutoSolver.autoSolveOutputs(mods, inp_map = inp_map)

XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="rw") do xf
    outputToXLSX(xf,outputs)
end

mod = Main.physics
mod = Main.physics.dynamics
::::MODULE Main.physics.dynamics::::

Func: fReact(:F_in,)
Inputs:
F_in = 5 N
Output: -5 N

Func: F(:m, :a)
Inputs:
m = 5 kg
a = 1 m s^-2
Output: 5 kg m s^-2

outputs = Dict{Any,Any}(:fReact=>-5 N,:F=>5 kg m s^-2)

mod = Main.physics.kinematics
::::MODULE Main.physics.kinematics::::

Func: a(:x0, :x, :v0, :v)
Inputs:
x0 = 1 m
x = 2 m
v0 = 1 m s^-1
vConstAcc1 = 1.7320508075688772 m s^-1
Output: 0.9999999999999998 m s^-2

Func: vConstAcc1(:x0, :x, :v0, :a)
Inputs:
x0 = 1 m
x = 2 m
v0 = 1 m s^-1
a = 1 m s^-2
Output: 1.7320508075688772 m s^-1

Func: xAbs(:x0, :x)
Inputs:
x0 = 1 m
x = 2 m
Output: 1 m

outputs = Dict{Any,Any}(:fReact=>-5 N,:a=>1.0 m s^-2,:F=>5 kg m s^-2,:vConstAcc1=>1.73205 m s^-1,:xAbs=>1 m)



# Archive

In [ ]:
mod = AutoSolver.modName(mods[1])

XLSX.openxlsx("AS_$(mod).xlsx", mode="w") do xf
    XLSX.addsheet!(xf)
    XLSX.addsheet!(xf)
    sh1 = xf[1]
    sh2 = xf[2]
    sh3 = xf[3]
    XLSX.rename!(sh1, "Inputs")
    XLSX.rename!(sh2, "Input Set")
    XLSX.rename!(sh3, "Outputs")
    
    sh1[1,1] = "Module"
    sh1[1,2] = "Function"
    sh1[1,3] = "Input Variable"
    sh1[1,4] = "Input"
    sh1[1,5] = "Units"
    
    sh2[1,1] = "Input Variable"
    sh2[1,2] = "Input"
    sh2[1,3] = "Units"
    
    sh3[1,1] = "Module"
    sh3[1,2] = "Function"
    sh3[1,3] = "Input Variables"
    sh3[1,4] = "Inputs"
    sh3[1,5] = "Units"
    sh3[1,6] = "Input Map"
    sh3[1,7] = "Output Map"
    sh3[1,8] = "Output"
    sh3[1,9] = "Units"
    
    r1 = 2
    r2 = 2
    r3 = 2
    new_input = 0
    
    for func in funcs 
        sh1[r1,1] = AutoSolver.modName(mods[2])
        sh1[r1,2] = string(func[1])
        
        for i = 1:length(func[2])
            sh1[r1,3] = string(func[2][i])
            r1+=1
            
            for j = 2:r2
                if string(sh2[j,3]) == string(func[2][i])
                    new_input = 0
                    break
                else
                    new_input = 1
                end
            end
            if new_input == 1
                sh2[r2,1] = string(func[2][i])
                r2+=1
            end
        end
    end
    
    for func in funcs
        inp_str = string(func[2][1])
        inp_map = string("[",func[2][1],"]")
        out_map = string("[",func[1],"]")
        
        sh3[r3,1] = AutoSolver.modName(mods[2])
        sh3[r3,2] = string(func[1])

        for i = 2:length(func[2])
            inp_str = string(inp_str,",",func[2][i])
            inp_map = string(inp_map,",","[",func[2][i],"]")
        end
        
        sh3[r3,3] = inp_str
        sh3[r3,6] = inp_map
        sh3[r3,7] = out_map
        sh3[r3,8] = "-"
        sh3[r3,9] = "-"
        r3+=1
    end
end

In [ ]:
inputs = [:x,:y,:z]
inp_str = string(inputs[1])
for i = 2:length(inputs)
    inp_str = string(inp_str,",",string(inputs[i]))
end

inp = [1.0,2.0,3]
units = ["m","km","N"]

XLSX.openxlsx("autosolve.xlsx", mode="w") do xf
    sh = xf[2]
    XLSX.rename!(sh, "Outputs")
    
    sh[1,1] = "Output Variable"
    sh[1,2] = "Output"
    sh[1,3] = "Units"
    sh[1,5] = "Input Variables"
    sh[1,6] = "Inputs"
    sh[1,7] = "Units"
    
    row = 2
    
    for out in outputs
        sh[row,1] = string(out[1])
        sh[row,2] = string(ustrip(out[2]))
        sh[row,3] = string(unit(out[2]))
        sh[row,5] = inp_str
        sh[row,6] = string(inp)
        sh[row,7] = string(units)
        row+=1
    end
end